# Temporal Unsupervised learning 

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jan  7 11:12:32 2021

@author: suhail
"""
# loading/processing the images  
import cv2
from IPython.display import Video
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.models import Model
import pickle

# clustering, dimension reduction, feature selection
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# learning classifiers
from mpl_toolkits.mplot3d import Axes3D

from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
import time
from scipy.io import savemat

# package for plotting dendograms 
from scipy.cluster.hierarchy import dendrogram, linkage


In [2]:
cwd = os.getcwd()
print(cwd)

F:\Projects\NIH_Organoid\Jup


## Load models and data


In [3]:
# CNN for feature extraction  
from keras.applications.vgg16 import VGG16 
model = VGG16()
CNN_model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

# import the spatial classifiers
pkl_filename = 'spatial_classifiers_22k.pkl'
SVM_model = pickle.load(open(pkl_filename, 'rb'))


## Define the functions we will use

In [4]:
def extract_features(file, CNN_model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
        # image_data_bw = img.max(axis=2)
        # non_empty_columns = np.where(image_data_bw.max(axis=0)>0)[0]
        # non_empty_rows = np.where(image_data_bw.max(axis=1)>0)[0]
        # cropBox = (min(non_empty_rows), max(non_empty_rows), min(non_empty_columns), max(non_empty_columns))
        # img_new = img[cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1 , :]
        # new_image = img.fromarray(img_new)
        # new_image.save('%s.png',file)
        # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
        # img = np.mean(img, axis=2)
        
    reshaped_img = img.reshape(1,224,224,3) 
    
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = CNN_model.predict(imgx, use_multiprocessing=True)
    
    return features

In [5]:
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:29]
    # plot each image in the cluster
    for index, file in enumerate(files):
        print(file)
        plt.subplot(10,10,index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

def getList(dict): 
    return list(dict.keys())

In [6]:
def nTreeClus(my_list, n = None, method = "All", ntree = 10, C = None):
    """ nTreeClus is a clustering method by Mustafa Gokce Baydogan and Hadi Jahanshahi.
    The method is suitable for clustering categorical time series (sequences). 
    You can always have access to the examples and description in 
    https://github.com/HadiJahanshahi/nTreeClus
    If you have any question about the code, you may email hadijahanshahi [a t] gmail . com
    
    prerequisites:
        numpy
        pandas
        sklearn
        scipy
    
    Args:
        my_list: a list of sequences to be clustered
        n: "the window length" or "n" in nTreecluss, you may provide it or it will be
            calculated automatically if no input has been suggested.
        method: 
            DT: Decision Tree
            RF: Random Forest
            All: both methods
        ntree: number of trees to be used in RF method. The defualt value is 10. 
             (Being too small has a bad effect on accuracy and being too large increases the complexity. 
              no less than 5 and no greater than 20.)
        C: number of clusters if it is not provided, it will be calculated for 2 to 10.

    Returns:
        'C_DT': "the optimal number of clusters with the aid of Decision Tree",
        'C_RF': "the optimal number of clusters with the aid of Random Forest",
        'Parameter n': the parameter of the nTreeClus (n) - either calculated or manually given
        'distance_DT': "sparse disance between sequences with the aid of Decision Tree",
        'distance_RF': "sparse disance between sequences with the aid of Random Forest",
        'labels_DT': "labels based on the optimal number of clusters using DT",
        'labels_RF': "labels based on the optimal number of clusters using RF".
            
            NOTE: in order to convert the distance output to a square distance matrix, 
                "scipy.spatial.distance.squareform" should be used.
                
    ## simple example with the output
    my_list = ['evidence','evident','provide','unconventional','convene']
    nTreeClusModel = nTreeClus(my_list, method = "All")
    # {'C_DT': 2,
    # 'C_RF': 2,
    # 'distance_DT': array([ 0.05508882,  0.43305329,  0.68551455,  0.43305329,  0.5       ,
    #         0.7226499 ,  0.5       ,  0.86132495,  0.75      ,  0.4452998 ]),
    # 'distance_RF': array([ 0.0809925 ,  0.56793679,  0.42158647,  0.57878823,  0.56917978,
    #         0.47984351,  0.54545455,  0.55864167,  0.71278652,  0.3341997 ]),
    # 'labels_DT': array([0, 0, 0, 1, 1]),
    # 'labels_RF': array([0, 0, 0, 1, 1])}
    
    
    """
    ############# pre processisng #################
    import numpy as np
    import pandas as pd
    from sklearn import preprocessing
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    import scipy.spatial.distance as ssd
    from scipy.cluster.hierarchy import linkage
    from sklearn.metrics import silhouette_score 
    from scipy import cluster

    
    sequence_sep = pd.DataFrame(my_list) # create a DataFrame out of the given list
    #sequence_sep = pd.DataFrame([list(x) for x in sequence[0]]) # separating the sequence
    sequence_sep[sequence_sep.shape[1]] = None # adding an empty column to the end of sequence

    if n is None:
        min_length = min( map(len, my_list) )
        total_avg = round(sum( map(len, my_list) ) / len(my_list)) # average length of strings
        n = min (round(total_avg**0.5) + 1, min_length - 1)
    
    if (n < 3):
        raise ValueError("Parameter n could not be less than 3. Remove the sequences with the length shorter than 3 and then re-run the function.")
    
    
    ############# matrix segmentation #################
    firstiteration = 1
    for i in range(len(my_list)):
        for j in range(sequence_sep.shape[1]):
            if (j < np.min(np.where(sequence_sep.iloc[i].isnull())) - n + 1): #check where is the position of the first "None"
                if (firstiteration==1):
                    seg_mat = pd.DataFrame(sequence_sep.iloc[i,j:j+n]).transpose()
                    seg_mat['OriginalMAT_element'] = i
                    seg_mat.columns = [np.arange(0,n+1)]
                    firstiteration+=1
                else:
                    temp = pd.DataFrame(sequence_sep.iloc[i,j:j+n]).transpose()
                    temp['OriginalMAT_element'] = i
                    temp.columns = [np.arange(0,n+1)]
                    seg_mat = seg_mat.append(temp)
    seg_mat.columns = np.append(np.arange(0,n-1),('Class','OriginalMAT_element')) # renaming the column indexes
    # dummy variable for DT and RF
    le = preprocessing.LabelEncoder()
    seg_mat.loc[:,'Class'] = le.fit_transform(seg_mat.loc[:,'Class']) #make Y to numbers
    #creating dummy columns for categorical data; one-hot encoding
    seg_mat = pd.get_dummies(seg_mat).reset_index(drop=True)

    
    xtrain = seg_mat.drop(labels=['OriginalMAT_element','Class'],axis=1)
    ytrain = seg_mat['Class']

    ############# nTreeClus method using DT #################        
    if (method == "All") or (method == "DT"):
        dtree = DecisionTreeClassifier()
        fitted_tree = dtree.fit(X=xtrain,y=ytrain)
        predictiontree = dtree.predict(xtrain)
        ### finding the terminal nodes.
        terminal_tree = fitted_tree.tree_.apply(xtrain.values.astype('float32'))  #terminal output
        terminal_output_tree  = pd.DataFrame(terminal_tree)
        terminal_output_tree ['OriginalMAT_element'] = seg_mat['OriginalMAT_element'].values
        #terminal_output_tree.drop(labels=0,axis=1,inplace=True)
        terminal_output_tree.columns = ['ter','OriginalMAT_element']
        terminal_output_tree_F = pd.crosstab(terminal_output_tree.OriginalMAT_element,terminal_output_tree.ter)
        Dist_tree_terminal_cosine = ssd.pdist(terminal_output_tree_F,metric='cosine')
        HC_tree_terminal_cosine = linkage(Dist_tree_terminal_cosine, 'ward')
        #finding the number of clusters
        if C is None:
            max_clusters = min(11,len(my_list))
            ress_sil = []
            for i in range(2,max_clusters):
                assignment_tree_terminal_cosine = cluster.hierarchy.cut_tree(HC_tree_terminal_cosine,i).ravel() #.ravel makes it 1D array.
                ress_sil.append((silhouette_score(ssd.squareform(Dist_tree_terminal_cosine),assignment_tree_terminal_cosine,metric='cosine').round(3)*1000)/1000)
            C = ress_sil.index(max(ress_sil)) + 2        
        # assigning the correct label
        optimal_cluster_tree = C
        assignment_tree_terminal_cosine = cluster.hierarchy.cut_tree(HC_tree_terminal_cosine,C).ravel() #.ravel makes it 1D array.
    
        
    ############# nTreeClus method using RF #################        
    if (method == "All") or (method == "RF"):
        np.random.seed(123)
        forest = RandomForestClassifier(n_estimators= ntree ,max_features = 0.36)
        fitted_forest = forest.fit(X=xtrain,y=ytrain)
        predictionforest = forest.predict(xtrain)
        ### Finding Terminal Nodes
        terminal_forest = fitted_forest.apply(xtrain) #terminal nodes access
        terminal_forest = pd.DataFrame(terminal_forest)
        #Adding "columnindex_" to the beginning of all  
        terminal_forest = terminal_forest.astype('str')
        for col in terminal_forest:
            terminal_forest[col] = '{}_'.format(col) + terminal_forest[col]
        terminal_forest.head()
        for i in range(terminal_forest.shape[1]):
            if i == 0:
                tempor = pd.concat( [seg_mat['OriginalMAT_element'] , terminal_forest[i]],ignore_index=True,axis=1)
                rbind_terminal_forest = tempor
            else:
                tempor = pd.concat( [seg_mat['OriginalMAT_element'] , terminal_forest[i]],ignore_index=True,axis=1)
                rbind_terminal_forest = pd.concat ([rbind_terminal_forest, tempor],ignore_index=True)

        rbind_terminal_forest.columns = ['OriginalMAT_element','ter']
        terminal_output_forest_F = pd.crosstab(rbind_terminal_forest.OriginalMAT_element,rbind_terminal_forest.ter)
        Dist_RF_terminal_cosine = ssd.pdist(terminal_output_forest_F,metric='cosine')
        HC_RF_terminal_cosine = linkage(Dist_RF_terminal_cosine, 'ward')
        #finding the number of clusters
        if C is None:
            max_clusters = min(11,len(my_list))
            ress_sil = []
            for i in range(2,max_clusters):
                assignment_RF_terminal_cosine = cluster.hierarchy.cut_tree(HC_RF_terminal_cosine,i).ravel() #.ravel makes it 1D array.
                ress_sil.append((silhouette_score(ssd.squareform(Dist_RF_terminal_cosine),assignment_RF_terminal_cosine,metric='cosine').round(3)*1000)/1000)
            C = ress_sil.index(max(ress_sil)) + 2        
        # assigning the correct label
        optimal_cluster_RF = C
        assignment_RF_terminal_cosine = cluster.hierarchy.cut_tree(HC_RF_terminal_cosine,C).ravel() #.ravel makes it 1D array.

    ############# output #################                
    if (method == "All"):
        return {"distance_DT":Dist_tree_terminal_cosine, "labels_DT":assignment_tree_terminal_cosine, "C_DT":optimal_cluster_tree, "distance_RF":Dist_RF_terminal_cosine, "labels_RF": assignment_RF_terminal_cosine, "C_RF":optimal_cluster_RF, "Parameter n":n}
    elif (method == "DT"):
        return {"distance_DT":Dist_tree_terminal_cosine, "labels_DT":assignment_tree_terminal_cosine, "C_DT":optimal_cluster_tree, "Parameter n":n}
    else: 
        return {"distance_RF":Dist_RF_terminal_cosine, "labels_RF":assignment_RF_terminal_cosine, "C_DT":optimal_cluster_RF, "Parameter n":n}


## Extract time series data from the simulations 

In [7]:
# uncomment this cell for ready-to-use sample (don't forget to comment the next cell)
"""
rootdir = r"F:/Projects/NIH_Organoid/Jup/simulations/"
os.chdir(rootdir)

with open('traces.pkl', 'rb') as pickle_load:
    my_list =pickle.load(pickle_load)
    
with open('names.pkl', 'rb') as pickle_load:
    image_names =pickle.load(pickle_load)
# print the array
print(len(my_list[3]))
"""

'\nrootdir = r"F:/Projects/NIH_Organoid/Jup/simulations/"\nos.chdir(rootdir)\n\nwith open(\'traces.pkl\', \'rb\') as pickle_load:\n    my_list =pickle.load(pickle_load)\n    \nwith open(\'names.pkl\', \'rb\') as pickle_load:\n    image_names =pickle.load(pickle_load)\n# print the array\nprint(len(my_list[3]))\n'

In [13]:
def extract_trace(path, CNN_model,SVM_model):
    # change the working directory to the path where the images are located
    os.chdir(path)

    # this list holds all the image filename
    images = []
    # creates a ScandirIterator aliased as files
    with os.scandir(path) as files:
      # loops through each file in the directory
        for file in files:
            if file.name.endswith('.png'): #or file.name.endswith('.jpg'):
              # adds only the image files to the images list
                images.append(file.name)
              
    dic_img = {}
    p = r"dump"
    traj_dist =[]
    labels = []
    # loop through each image in the dataset
    only_class_labels = []
    for image in sorted(images):
        # try to extract the features and update the dictionary
        image_name= os.path.splitext(image)[0]
        try:
            feat = extract_features(image,CNN_model)
            label = SVM_model.predict(feat.reshape(1,-1))
            arr = SVM_model.decision_function(feat.reshape(1,-1)).reshape(-1,1)
            traj_dist.append(arr.tolist()) 
            labels.append(int(label[0]))
            dic_img[image_name]= (labels,traj_dist,feat.tolist())
        # if something fails, save the extracted features as a pickle file (optional)
        except Exception as e: 
            print(e)
            print('failed')
            with open(p,'wb') as file:
                pickle.dump(data,file)

    # get a list of the filenames
    filenames = np.array(list(dic_img.keys()))
    
    return filenames, labels,traj_dist, dic_img 
   

In [14]:
#are you sure?
rootdir = r"F:/Projects/NIH_Organoid/Jup/simulations"

traces=[]
data_run={}
image_names=[]
labels_run_im = []
run_names_full=[]
run_names =[]
traj_dist =[]
counter =0
for root, subdirs, files in os.walk(rootdir):
    for run_name_full in sorted(subdirs):
        path = os.path.join(root, run_name_full)
        print(path)
        run_name = ''.join(e for e in run_name_full if e.isalnum())
        image_names_temp, labels_temp,traj_dist_temp,data_temp = extract_trace(path, CNN_model,SVM_model)
        if len(labels_temp)>=49:
            run_names.append(run_name)
            run_names_full.append(run_name_full)
            image_names.append(image_names_temp[:49])
            labels_run_im.append(labels_temp[:49])  
            traj_dist.append(traj_dist_temp[:49])     
            data_run[run_name]= (run_name_full,image_names_temp[:49],labels_temp[:49],traj_dist_temp[:49],data_temp)
            

os.chdir(rootdir)
print('image_names', len(image_names[0]))
print('labels_run_im', len(labels_run_im[0]))
print('traj_dist', len(traj_dist[0]))


with open('run_names_full.pkl', 'wb') as pickle_file:
    pickle.dump(run_names_full, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('image_names.pkl', 'wb') as pickle_file:
    pickle.dump(image_names, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('labels_run_im.pkl', 'wb') as pickle_file:
    pickle.dump(labels_run_im, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('traj_dist.pkl', 'wb') as pickle_file:
    pickle.dump(traj_dist, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('data_run.pkl', 'wb') as pickle_file:
    pickle.dump(data_run, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
    
# save a copy for matlab use
savemat('run_names_full.mat',{"run_names_full":run_names_full})
savemat('image_names.mat',{"image_names":image_names})
savemat('labels_run_im.mat',{"labels_run_im":labels_run_im})
savemat('traj_dist.mat', {"labels_run_im":labels_run_im})
savemat('data_run.mat', data_run)

F:/Projects/NIH_Organoid/Jup/simulations\s (1)
F:/Projects/NIH_Organoid/Jup/simulations\s (10)
F:/Projects/NIH_Organoid/Jup/simulations\s (100)
F:/Projects/NIH_Organoid/Jup/simulations\s (101)
F:/Projects/NIH_Organoid/Jup/simulations\s (102)
F:/Projects/NIH_Organoid/Jup/simulations\s (103)
F:/Projects/NIH_Organoid/Jup/simulations\s (104)
F:/Projects/NIH_Organoid/Jup/simulations\s (105)
F:/Projects/NIH_Organoid/Jup/simulations\s (106)
F:/Projects/NIH_Organoid/Jup/simulations\s (107)
F:/Projects/NIH_Organoid/Jup/simulations\s (108)
F:/Projects/NIH_Organoid/Jup/simulations\s (109)
F:/Projects/NIH_Organoid/Jup/simulations\s (11)
F:/Projects/NIH_Organoid/Jup/simulations\s (110)
F:/Projects/NIH_Organoid/Jup/simulations\s (111)
F:/Projects/NIH_Organoid/Jup/simulations\s (112)
F:/Projects/NIH_Organoid/Jup/simulations\s (113)
F:/Projects/NIH_Organoid/Jup/simulations\s (114)
F:/Projects/NIH_Organoid/Jup/simulations\s (115)
F:/Projects/NIH_Organoid/Jup/simulations\s (116)
F:/Projects/NIH_Organoid

F:/Projects/NIH_Organoid/Jup/simulations\s (250)
F:/Projects/NIH_Organoid/Jup/simulations\s (251)
F:/Projects/NIH_Organoid/Jup/simulations\s (252)
F:/Projects/NIH_Organoid/Jup/simulations\s (253)
F:/Projects/NIH_Organoid/Jup/simulations\s (254)
F:/Projects/NIH_Organoid/Jup/simulations\s (255)
F:/Projects/NIH_Organoid/Jup/simulations\s (256)
F:/Projects/NIH_Organoid/Jup/simulations\s (257)
F:/Projects/NIH_Organoid/Jup/simulations\s (258)
F:/Projects/NIH_Organoid/Jup/simulations\s (259)
F:/Projects/NIH_Organoid/Jup/simulations\s (26)
F:/Projects/NIH_Organoid/Jup/simulations\s (260)
F:/Projects/NIH_Organoid/Jup/simulations\s (261)
F:/Projects/NIH_Organoid/Jup/simulations\s (262)
F:/Projects/NIH_Organoid/Jup/simulations\s (263)
F:/Projects/NIH_Organoid/Jup/simulations\s (264)
F:/Projects/NIH_Organoid/Jup/simulations\s (265)
F:/Projects/NIH_Organoid/Jup/simulations\s (266)
F:/Projects/NIH_Organoid/Jup/simulations\s (267)
F:/Projects/NIH_Organoid/Jup/simulations\s (268)
F:/Projects/NIH_Organ

F:/Projects/NIH_Organoid/Jup/simulations\s (401)
F:/Projects/NIH_Organoid/Jup/simulations\s (402)
F:/Projects/NIH_Organoid/Jup/simulations\s (403)
F:/Projects/NIH_Organoid/Jup/simulations\s (404)
F:/Projects/NIH_Organoid/Jup/simulations\s (405)
F:/Projects/NIH_Organoid/Jup/simulations\s (406)
F:/Projects/NIH_Organoid/Jup/simulations\s (407)
F:/Projects/NIH_Organoid/Jup/simulations\s (408)
F:/Projects/NIH_Organoid/Jup/simulations\s (409)
F:/Projects/NIH_Organoid/Jup/simulations\s (41)
F:/Projects/NIH_Organoid/Jup/simulations\s (410)
F:/Projects/NIH_Organoid/Jup/simulations\s (411)
F:/Projects/NIH_Organoid/Jup/simulations\s (412)
F:/Projects/NIH_Organoid/Jup/simulations\s (413)
F:/Projects/NIH_Organoid/Jup/simulations\s (414)
F:/Projects/NIH_Organoid/Jup/simulations\s (415)
F:/Projects/NIH_Organoid/Jup/simulations\s (416)
F:/Projects/NIH_Organoid/Jup/simulations\s (417)
F:/Projects/NIH_Organoid/Jup/simulations\s (418)
F:/Projects/NIH_Organoid/Jup/simulations\s (419)
F:/Projects/NIH_Organ

C:\Users\suhai\anaconda3\envs\organoid\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [138]:
rootdir = r"F:/Projects/NIH_Organoid/Jup/images"
os.chdir(rootdir)
pkl_filename = "selected_features.pkl"
ind_selected_feat = pickle.load(open(pkl_filename, 'rb'))
rootdir = r"F:/Projects/NIH_Organoid/Jup/simulations"
os.chdir(rootdir)
feat_latest ={}
feat_red ={}
foot ={}
for key1 in data_run.keys():
    foot = data_run [key1][4]
    feat_latest ={}
    for index, key2 in enumerate(foot.keys()):
        TEMPP = foot[key2] 
        feat_latest[key2] =np.array(TEMPP[2][0])[ind_selected_feat.astype(int)]
    feat_red[key1]=feat_latest
    
savemat('all_feat_select.mat', feat_red)    


In [137]:
feat_red[key1]

{'plot_00000': array([0.        , 0.60254174, 0.71538162, 0.        , 0.801202  ,
        0.        , 0.        , 0.        , 0.        , 4.28282213,
        0.        , 3.15149641, 0.        , 2.9169383 , 0.        ,
        0.        , 2.63220263, 0.54680598, 1.64621234, 0.        ]),
 'plot_00050': array([0.49902859, 0.        , 1.34267497, 0.        , 2.69227552,
        1.08625746, 0.49356878, 0.        , 0.        , 2.00433493,
        0.        , 4.2838974 , 0.        , 2.75030994, 1.31952322,
        0.        , 1.16086721, 0.        , 0.7991842 , 0.        ]),
 'plot_00100': array([0.        , 0.        , 0.01559415, 0.        , 1.81920481,
        0.21455809, 0.25929931, 0.        , 0.        , 2.39654207,
        0.91862226, 3.82692027, 0.        , 3.52026749, 0.68943304,
        0.        , 2.00538421, 0.23595974, 0.        , 0.        ]),
 'plot_00150': array([0.        , 0.        , 0.        , 0.        , 1.56978321,
        0.18685442, 0.01741904, 0.        , 0.        

In [ ]:
rootdir = r"F:/Projects/NIH_Organoid/Jup/simulations"
os.chdir(rootdir)
pkl_filename = 'run_names_full.pkl'
run_names_full = pickle.load(open(pkl_filename, 'rb'))
pkl_filename = 'image_names.pkl'
image_names = pickle.load(open(pkl_filename, 'rb'))
pkl_filename = 'labels_run_im.pkl'
labels_run_im = pickle.load(open(pkl_filename, 'rb'))
pkl_filename = 'traj_dist.pkl'
traj_dist = pickle.load(open(pkl_filename, 'rb'))
pkl_filename = 'data_run.pkl'
data_run = pickle.load(open(pkl_filename, 'rb'))
rootdir = r"F:/Projects/NIH_Organoid/Jup/images"
os.chdir(rootdir)
pkl_filename = "selected_features.pkl"
ind_selected_feat = pickle.load(open(pkl_filename, 'rb'))
rootdir = r"F:/Projects/NIH_Organoid/Jup/simulations"
os.chdir(rootdir)

In [12]:
rootdir = r"F:/Projects/NIH_Organoid/Jup/images"
os.chdir(rootdir)
pkl_filename = "selected_features.pkl"
ind_selected_feat = pickle.load(open(pkl_filename, 'rb'))
ind_selected_feat
savemat('ind_selected_feat.mat', {"foo":ind_selected_feat})

In [ ]:
#  parallel compute [not finished]
"""
rootdir = r"F:/Projects/NIH_Organoid/Jup/simulations/"

def get_traces(root, subdirs, files):
    traces={}
    names ={}
    data={}
    my_list = []
    image_names=[]
    counter =0
    for run_name in sorted(subdirs):
        path = os.path.join(root, run_name)
        print(path)
        names[run_name], traces_temp,data[run_name] = extract_trace(path, CNN_model,SVM_model)
        if traces_temp.shape >=(40,): 
            traces[run_name] = traces_temp            
            my_list.append(traces_temp[:,4096].tolist())
            image_names.append(run_name)
            counter = counter +1 
    return my_list       
                
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())
pool = mp.Pool(mp.cpu_count())
my_list = [pool.apply(get_traces, args=(root, subdirs, files)) for root, subdirs, files in os.walk(rootdir)]
pool.close()    
os.chdir(rootdir)
"""

In [ ]:
#simple example with the output 
nTreeClusModel = nTreeClus(labels_run_im, method = "All",ntree = 10, C=4)
nTreeClusModel

In [ ]:
HC_tree_terminal_cosine = linkage(nTreeClusModel["distance_DT"], 'ward')
fig = plt.figure(figsize=(25, 10))
ax = fig.add_subplot(1, 1, 1)
dendrogram(HC_tree_terminal_cosine,labels=run_names, ax=ax)
ax.tick_params(axis='x', which='major', labelsize=10)
ax.tick_params(axis='y', which='major', labelsize=15)
plt.show()

HC_RF_terminal_cosine = linkage(nTreeClusModel["distance_RF"], 'ward')
fig = plt.figure(figsize=(25, 10))
ax = fig.add_subplot(1, 1, 1)
dendrogram(HC_RF_terminal_cosine,labels=run_names, ax=ax)
ax.tick_params(axis='x', which='major', labelsize=10)
ax.tick_params(axis='y', which='major', labelsize=15)
plt.show()

In [ ]:
labels_run=nTreeClusModel["labels_DT"]
all = list(zip(run_names,run_names_full,labels_run_im,traj_dist, labels_run))
# print(len(all))
# print(all[0])

savemat('all_LTI.mat',dict(names =run_names, labels_run_im =labels_run_im,labels_run = labels_run, traj_dist =traj_dist ))
savemat('all.mat', dict(all=all))
with open('all_LTI.pkl', 'wb') as pickle_file:
    pickle.dump(all, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
    
#savemat('data_all.mat',data)
#with open('name_trace_label.pkl', 'rb') as pickle_load:
#    b =pickle.load(pickle_load)

In [ ]:
labels_run

In [ ]:
def getDuplicatesWithCount(names,listOfLists):
    ''' Get frequency count of duplicate elements in the given list '''
    dictOfLists = dict()
    dictOfFormulae = dict()
    i=0
    # Iterate over each element in list
    for listOfElems in listOfLists:
        dictOfElems = dict()
        for elem in  listOfElems:
        # If element exists in dict then increment its value else add it in dict
            if elem in dictOfElems:
                dictOfElems[elem] += 1
            else:
                dictOfElems[elem] = 1  
    # Filter key-value pairs in dictionary. Keep pairs whose value is greater than 1 i.e. only duplicate elements from list.
        dictOfElems = { key:value for key, value in dictOfElems.items() if value > 0}
        time2=0
        dictOfFormula = dict()
        for key, value in dictOfElems.items():
            time1= time2
            time2=time1+500*value    
            dictOfFormula[key] = time1,time2    
        dictOfLists[names[i]] = dictOfElems
        dictOfFormulae[names[i]]=dictOfFormula
        i+=1
    # Returns a dict of duplicate elements and thier frequency count
    return dictOfLists,dictOfFormulae



In [ ]:
dictOfLists,dictOfFormulae=getDuplicatesWithCount(run_names,labels_run_im)

#all_time_cls = list(zip(dictOfLists.keys,dictOfFormulae, labels))
ex = run_names[100]
print('for run with ID: ', ex)

print(dictOfLists[ex])
print(dictOfFormulae[ex])


In [ ]:
def getDuplicatestime(names,listOfLists,time_cls):
    ''' Get frequency count of duplicate elements all lists '''
    dictOfLists = dict()
    dictOfFormulae = dict()
    dictOftimesteps = dict()
    dictOftimecls =dict()
    dictOftimecls_names =dict()

    count =0
    for j in range(3):
        listOfnames=[]
        for i in range(22):
            dictOftimesteps[i*500] = []
            # Iterate over all lists
            count2= 0
            for listOfElems in listOfLists:
            # If element exists in dict then increment its value else add it in dict
                if (time_cls[count2]==count) and (listOfElems[i] not in dictOftimesteps[i*500]):
                    dictOftimesteps[i*500].append(int(listOfElems[i]))
                if (time_cls[count2]==count) and (names[count2] not in listOfnames):
                    listOfnames.append(names[count2])
                count2+=1
        dictOftimecls[count] = dict(dictOftimesteps)
        dictOftimecls_names[count]= listOfnames
        count+=1
    # Returns a dict of duplicate elements and thier frequency count at each time 
    return dictOftimecls,dictOftimecls_names

In [ ]:
dictOftimes,dictOftimecls_names = getDuplicatestime(run_names,labels_run_im,labels_run)
class_id =1
print(dictOftimes[class_id])
print(dictOftimecls_names[class_id])


In [ ]:
def view_class(class_members):
    plt.figure(figsize = (25,25));
    # gets the list of filenames for a cluster
    folders = class_members
    
    # only allow up to 30 images to be shown at a time
    if len(folders) > 5:
        print(f"Clipping cluster size from {len(folders)} to 5")
        folders = folders[:5]
    
    for index, folder in enumerate(folders):
        video_name = folder+'.avi'
        images = [img for img in os.listdir(folder) if img.endswith(".png")]
        image_last = os.path.join(folder, images[-1])
        image_first = os.path.join(folder, images[0])
        frame = cv2.imread(os.path.join(folder, images[0]))
        height, width, layers = frame.shape

        video = cv2.VideoWriter(video_name, 0, 1, (width,height))
        
        for image in sorted(images):
            video.write(cv2.imread(os.path.join(folder, image)))
        cv2.destroyAllWindows()
        video.release()
        Video(video_name)
        print(image_last)
        plt.subplot(10,10,index+1);
        img = load_img(image_last)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')
#         plt.subplot(10,10,5+index+1);
#         img = load_img(image_first)
#         img = np.array(img)
#         plt.imshow(img)
        plt.axis('off')

In [ ]:
run_names_full

In [ ]:
rootdir = r"F:/Projects/NIH_Organoid/Jup/simulations/"
os.chdir(rootdir)
count =0
num_cls = 3
for i in range(num_cls):
    Folders = dictOftimecls_names[i]
    if len(Folders) > 2:
            print(f"Clipping cluster size from {len(Folders)} to 1")
            Folders = Folders[:1]
    for folder_name in Folders:
        for full_name in run_names_full:
            if folder_name == ''.join(e for e in full_name if e.isalnum()):
                image_folder = full_name
                break

        video_name = folder_name+'_'+str(i)+'.avi'

        images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
        frame = cv2.imread(os.path.join(image_folder, images[0]))
        height, width, layers = frame.shape

        video = cv2.VideoWriter(video_name, 0, 1, (width,height))

        for image in sorted(images):
            video.write(cv2.imread(os.path.join(image_folder, image)))
        cv2.destroyAllWindows()
        video.release()

        files = image_names[0]
        files = files[[0,5,10,15,20,25,30,35]]
        cwd = os.getcwd()
        dir_name = full_name
        for index, file in enumerate(files):
            count =count+1
            file_dir =  os.path.join(cwd,dir_name, files[index]+"."+'png' )
#             print(file_dir)
            plt.subplot(num_cls,9,count+1);
            img = load_img(file_dir)
            img = np.array(img)
            plt.imshow(img)
            plt.axis('off')



In [ ]:

files

In [ ]:
#to do: 
# - show videos 
cwd = os.getcwd()
print(cwd)
rootdir = r"F:/Projects/NIH_Organoid/Jup/simulations/"
os.chdir(rootdir)
for class_id in range(2):
    print(dictOftimes[class_id])
    view_class(dictOftimecls_names[class_id])

In [ ]:
dictOftimecls_names

# Logical lens


In [ ]:
import pandas as pd  # To install pandas, `pip install pandas`
import glob
import matplotlib.pyplot as plt
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from sklearn import preprocessing
from sklearn.mixture import GaussianMixture

# Data Science
from traces import TimeSeries
from traces import domain as Domain

# Utilities Library
import funcy as fn

# Path utilities
from pathlib import Path

# Python STL library written for this project: github.com/mvcisback/py-stl
import mtl
# Parallelize The projections
import multiprocessing

from IPython.display import display

In [ ]:
# fine the files in the path
path = r'F:\Projects\TLI\Marcell\LogicalLens-master\example_data\toy_car_speeds'
all_files = glob.glob(path + "/*.csv")

li = []
data =[]
def load_data(path):
    speeds = pd.read_csv(path).Y
    return list(zip(speeds.index, speeds))
for filename in all_files:
    df = load_data(filename)
    data.append(df)
    print(filename)

In [ ]:
#!conda install -c bioconda fastdtw
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

In [ ]:
# DTW distance measure function
def dtw_dist(x, y):
    return fastdtw(x['Y'].sample(0.1), y['Y'].sample(0.1), dist=euclidean)[0]
# PSTL 
def slow_down_lens(data):
    def spec(params):
        h, tau = params
        tau *= 20
        return all(speed < h for t, speed in data if t >= tau)
    return spec

# distances + 
def dist(x, y):
    return np.linalg.norm(x - y)


def val_to_vec(val):
    return np.array([x for _, x in sorted(list(val.items()))])

def project(x):
    order = ("a?", "tau?")
    ranges = {"a?": (-3, 3), "tau?": (10.0001, 1000)}
    polarity = {"a?": False, "tau?": True} # Monotonically Increasing or Decreasing
    return stl.lex_param_project(
        phi, x, order=order, ranges=ranges, polarity=polarity)

DISTS = np.vstack(list(map(lambda x: val_to_vec(project(x)), traces)))
stl_sigma = np.var(DISTS, axis=0)
stl_mu = np.mean(DISTS, axis=0)

def stl_dist(x, y):
    return dist(*map(lambda a: (val_to_vec(project(a)) - stl_mu)/stl_sigma, (x, y)))

In [ ]:
# define the lens as class logicLens
from logical_lens import LogicalLens
lens = LogicalLens(n=2, lens=slow_down_lens)

In [ ]:
# List of rectangles
recs = lens.boundary(data[0], approx=True, tol=1e-4) 
print(recs)

In [ ]:
# Compute Logical Distances.
d = lens.dist(data[0], data[1])
A = lens.adj_matrix(data=data)  # Compute full adjacency matrix.
print(A)

In [ ]:
dists = squareform(np.array(A))
linkage_matrix = linkage(dists,'ward')
dendrogram(linkage_matrix, labels=["0", "1", "2","3"])
plt.title("test")
plt.show()


In [ ]:
points = [
    (0, 1),   # Reference line intersecting origin and (0, 1)
    (1, 0.3)  # ..                                     (1, 0.3)
]
f = lens.projector(points)  # Note, will project to -1 or 2 if no intersection
                            # is found.
Y = [f(d) for d in data]
print(Y)



In [ ]:
## Project onto 2 random lines.
f2 = lens.random_projector(2)
Y = [f2(d) for d in data]
X = np.vstack([f2(d) for d in data])  # collect projected data into a matrix.
print('X = ')
print(X)


In [ ]:
# Throw out data that had no intersections (and thus no slow downs).
#intersects = (data != 2).X[0] * (data != 2).X[1]
#X = X[intersects]

# Learn a guassian mixture model
model = GaussianMixture(4)
model.fit(X)

labels = np.array([model.predict(x.reshape(1,2))[0] for x in X])
print(labels)

In [ ]:
X_np =np.array(X)
print(X_np)
plt.scatter(X_np[:,0],X_np[:,1])
plt.show()

In [ ]:
#ref = data[3] # toy_data[0]
#   dists = [lens.dist(ref, d) for d in data]

In [ ]:
slow_downs = data  # data identified as slow downs

f1 = lens.projector([(0.5, 1)])
f2 = lens.projector([(1, 0.2)])
X1 = np.vstack([f1(d) for d in slow_downs])
X2 = np.vstack([f2(d) for d in slow_downs])

box1 = X1.min(axis=0), X1.max(axis=0)  # (0.25, 0.55), (0.38, 0.76)
box2 = X2.min(axis=0), X2.max(axis=0)  # (0.35, 0.17), (0.62, 0.31)
print(box1)
print(box2)
